In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from resnet import ResNet34

In [12]:
num_epochs = 25
batch_size = 128
learning_rate = 0.001
model_path = "resnet34_cifar10.pth"

In [13]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [14]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
model = ResNet34(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
train_losses = []
def train():
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
                train_losses.append(running_loss / 100)
                running_loss = 0.0
        evaluate()  

    
    torch.save(model.state_dict(), model_path)
    print(f'Model saved to {model_path}')

In [18]:
val_losses = []
val_accuracies = []
def evaluate():
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(test_loader)
    accuracy = 100 * correct / total
    val_losses.append(val_loss)
    val_accuracies.append(accuracy)

    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [ ]:
if __name__ == "__main__":
    train()
    evaluate()

In [19]:
import numpy as np
print(f"num_epochs: {np.array(num_epochs).shape}")
print(f"train_losses: {np.array(train_losses).shape}")
print(f"val_losses: {np.array(val_losses).shape}")
print(f"val_accuracies: {np.array(val_accuracies).shape}")


num_epochs: ()
train_losses: (0,)
val_losses: (0,)
val_accuracies: (0,)
